In [1]:
using TSSM
include("time_propagators.jl")

step! (generic function with 5 methods)

# Test problem: nonlinear Schrödinger equation with time dependent potential

The soliton
$$\psi_\mathrm{ex}(x,t)=\frac{a\mathrm{e}^{\frac{1}{2}\mathrm{i}((a^2-b^2)t-bx)}}{\cosh(a(bt+x-c))}$$ 
is the exact solution of the cubic nonlinear Schrödinger
$$\mathrm{i}\frac{\partial}{\partial t}\psi(x,t)=-\frac{1}{2}\Delta\psi(x,t)+\kappa|\psi(x,t)|^2\psi(x,t),
\quad \psi(x,0) = \psi_\mathrm{ex}(x,0),$$
where $\kappa=-1$, and $a,b,c\in\mathbb{R}$ are arbitrarily chosen parameters. 

It follows that 
the exact solution of the *nonlinear* Schrödinger equation
$$\mathrm{i}\frac{\partial}{\partial t}\psi(x,t)=-\frac{1}{2}\Delta\psi(x,t)+V_\alpha(x,t)\psi(x,t)+\alpha\kappa|\psi(x,t)|^2\psi(x,t),\quad\psi(x,0) = \psi_\mathrm{ex}(x,0),$$
with *time dependent potential*
$$V_\alpha(x,t)= (1-\alpha)\kappa|\psi_\mathrm{ex}(x,t)|^2=\frac{(1-\alpha)\kappa a^2}{\cosh(a(bt+x-c))^2}$$
and, e.g., $\alpha=\frac{1}{2}$ is again given by $\psi_\mathrm{ex}(x,t)$.

We use this time-dependent problem to verify the correct implementation of various time propagation methods

In [2]:
# exact solution
function soliton(x, t)
    a = 2.0
    b = 1.0
    c = 0.0    
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [3]:
nx = 1024
xmin = -16
xmax = +16
V(x,t) = -0.5*abs(soliton(x,t)).^2
m = Schroedinger1D(nx, xmin, xmax, potential_t=V, cubic_coupling=-0.5)
psi = wave_function(m)
psi_ref = wave_function(m)

TSSM.WfSchroedinger1D{Float64}(Ptr{Void} @0x00000000046a6230, TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x0000000004c820b0))

In [4]:
t0 = 0.0
tend = 1.0
set!(psi_ref, soliton, tend) # exact reference solution

# One-step methods

## Explicit Runge-Kutta
### Classical Runge-Kutta scheme of order 4

In [5]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:rk4)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01         NaN
  2   1.250e-01         NaN         NaN    NaN
  3   6.250e-02         NaN         NaN    NaN
  4   3.125e-02         NaN         NaN    NaN
  5   1.563e-02         NaN         NaN    NaN
  6   7.813e-03         NaN         NaN    NaN
  7   3.906e-03         NaN         NaN    NaN
  8   1.953e-03         NaN         NaN    NaN
  9   9.766e-04         NaN         NaN    NaN
 10   4.883e-04   4.935e-13         NaN    NaN


## Splitting Method
### 5-step composition of order 4 (Suzuki)

In [6]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
method = SplittingMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   5.413e-02
  2   1.250e-01   8.118e-03   2.407e+00   2.74
  3   6.250e-02   5.281e-04   2.949e+01   3.94
  4   3.125e-02   1.003e-05   4.064e+03   5.72
  5   1.563e-02   2.574e-07   8.996e+02   5.28
  6   7.813e-03   1.548e-08   5.437e+00   4.06
  7   3.906e-03   9.598e-10   4.395e+00   4.01
  8   1.953e-03   5.987e-11   4.187e+00   4.00
  9   9.766e-04   3.792e-12   3.653e+00   3.98
 10   4.883e-04   1.089e-12   9.914e-07   1.80


### 3-step composition of order 4 (Yoshida)

In [7]:
set!(psi, soliton, t0)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
a, b = get_coeffs_composition(g)
method = SplittingMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   1.709e-01
  2   1.250e-01   2.148e-02   1.083e+01   2.99
  3   6.250e-02   1.270e-03   1.038e+02   4.08
  4   3.125e-02   4.294e-05   9.731e+02   4.89
  5   1.563e-02   2.549e-06   5.830e+01   4.07
  6   7.813e-03   1.614e-07   3.945e+01   3.98
  7   3.906e-03   1.013e-08   4.224e+01   3.99
  8   1.953e-03   6.336e-10   4.317e+01   4.00
  9   9.766e-04   3.962e-11   4.331e+01   4.00
 10   4.883e-04   2.624e-12   2.439e+01   3.92


## Composition Method, B-part: implizit midpoint rule
### 5-step composition of order 4 (Suzuki), 3 fixed-point iterations

In [8]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
method = CompositionMethod(g,2)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   3.357e-01
  2   1.250e-01   5.777e-02   1.134e+01   2.54
  3   6.250e-02   7.520e-03   2.619e+01   2.94
  4   3.125e-02   9.324e-04   3.182e+01   3.01
  5   1.563e-02   1.155e-04   3.193e+01   3.01
  6   7.813e-03   1.436e-05   3.130e+01   3.01
  7   3.906e-03   1.790e-06   3.075e+01   3.00
  8   1.953e-03   2.234e-07   3.040e+01   3.00
  9   9.766e-04   2.790e-08   3.020e+01   3.00
 10   4.883e-04   3.486e-09   3.012e+01   3.00


### 4 fixed-point iterations

In [9]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
method = CompositionMethod(g,3)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   4.153e-02
  2   1.250e-01   5.614e-03   2.273e+00   2.89
  3   6.250e-02   4.207e-04   1.335e+01   3.74
  4   3.125e-02   8.396e-06   2.650e+03   5.65
  5   1.563e-02   2.256e-07   5.992e+02   5.22
  6   7.813e-03   1.433e-08   3.435e+00   3.98
  7   3.906e-03   9.139e-10   3.343e+00   3.97
  8   1.953e-03   5.785e-11   3.549e+00   3.98
  9   9.766e-04   3.847e-12   2.274e+00   3.91
 10   4.883e-04   1.141e-12   7.299e-07   1.75


### 3-step composition of order 4 (Yoshida), 3 fixed-point iterations

In [10]:
set!(psi, soliton, t0)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
method = CompositionMethod(g,3)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   9.449e-01
  2   1.250e-01   2.432e-01   1.427e+01   1.96
  3   6.250e-02   1.409e-02   1.251e+03   4.11
  4   3.125e-02   4.193e-04   1.793e+04   5.07
  5   1.563e-02   1.015e-05   5.037e+04   5.37
  6   7.813e-03   3.930e-07   3.020e+03   4.69
  7   3.906e-03   3.126e-08   1.951e+01   3.65
  8   1.953e-03   2.312e-09   3.495e+01   3.76
  9   9.766e-04   1.569e-10   7.556e+01   3.88
 10   4.883e-04   1.050e-11   8.753e+01   3.90


## Exponential Runge-Kutta 
### Krogstad scheme of order 4

In [11]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:krogstad)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   3.870e-02
  2   1.250e-01   4.560e-03   2.787e+00   3.09
  3   6.250e-02   3.267e-04   1.240e+01   3.80
  4   3.125e-02   2.040e-05   2.148e+01   4.00
  5   1.563e-02   1.245e-06   2.410e+01   4.03
  6   7.813e-03   7.638e-08   2.338e+01   4.03
  7   3.906e-03   4.721e-09   2.217e+01   4.02
  8   1.953e-03   2.933e-10   2.128e+01   4.01
  9   9.766e-04   1.827e-11   2.075e+01   4.00
 10   4.883e-04   1.157e-12   1.755e+01   3.98


### Strehmel-Weiner scheme of order 4

In [12]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:strehmel_weiner)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   5.246e-02
  2   1.250e-01   5.453e-03   4.854e+00   3.27
  3   6.250e-02   3.831e-04   1.573e+01   3.83
  4   3.125e-02   2.401e-05   2.482e+01   4.00
  5   1.563e-02   1.475e-06   2.748e+01   4.03
  6   7.813e-03   9.086e-08   2.695e+01   4.02
  7   3.906e-03   5.631e-09   2.589e+01   4.01
  8   1.953e-03   3.503e-10   2.510e+01   4.01
  9   9.766e-04   2.184e-11   2.457e+01   4.00
 10   4.883e-04   1.376e-12   2.224e+01   3.99


### Runge-Kutta-Lawson scheme of order 4

In [13]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:lawson)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   1.144e-01
  2   1.250e-01   1.482e-02   6.824e+00   2.95
  3   6.250e-02   9.790e-04   5.134e+01   3.92
  4   3.125e-02   4.565e-05   2.072e+02   4.42
  5   1.563e-02   2.844e-06   4.861e+01   4.00
  6   7.813e-03   1.796e-07   4.482e+01   3.98
  7   3.906e-03   1.127e-08   4.701e+01   3.99
  8   1.953e-03   7.051e-10   4.787e+01   4.00
  9   9.766e-04   4.405e-11   4.862e+01   4.00
 10   4.883e-04   2.714e-12   5.600e+01   4.02


# Multistep methods

In [14]:
type ExactSolution <: TimePropagationMethod
end
function step!(m::ExactSolution, psi::WaveFunction, 
         t0::Real, dt::Real, steps::Int, step::Int)
    set!(psi, soliton, t0+(step+1)*dt)
end    

step! (generic function with 6 methods)

## Exponential multistep
### exact evaluation of the integral; 4 steps; exact starting values; no correction step

In [15]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=0, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   4.888e-01
  2   1.250e-01   6.979e-02   2.397e+01   2.81
  3   6.250e-02   4.556e-03   2.508e+02   3.94
  4   3.125e-02   2.948e-04   2.600e+02   3.95
  5   1.563e-02   1.849e-05   3.036e+02   3.99
  6   7.813e-03   1.153e-06   3.142e+02   4.00
  7   3.906e-03   7.193e-08   3.140e+02   4.00
  8   1.953e-03   4.490e-09   3.121e+02   4.00
  9   9.766e-04   2.804e-10   3.110e+02   4.00
 10   4.883e-04   1.742e-11   3.262e+02   4.01


### exact evaluation of the integral; 3 steps; exact starting values; one correction step

In [16]:
set!(psi, soliton, t0)
method = ExponentialMultistep(3, iters=1, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   1.382e-01
  2   1.250e-01   2.286e-02   5.046e+00   2.60
  3   6.250e-02   2.362e-03   2.073e+01   3.27
  4   3.125e-02   1.645e-04   1.003e+02   3.84
  5   1.563e-02   1.031e-05   1.705e+02   4.00
  6   7.813e-03   6.355e-07   1.876e+02   4.02
  7   3.906e-03   3.930e-08   1.838e+02   4.02
  8   1.953e-03   2.441e-09   1.775e+02   4.01
  9   9.766e-04   1.520e-10   1.735e+02   4.01
 10   4.883e-04   9.385e-12   1.885e+02   4.02


### Gauss quadrature of order 4 for the  integral; 3 steps; exact starting values; one correction step

In [17]:
set!(psi, soliton, t0)
gauss4 = QuadratureRule([(3-sqrt(3))/6, (3+sqrt(3))/6],[0.5,0.5])
method = ExponentialMultistep(3, iters=1, quadrature=gauss4, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   2.367e-01
  2   1.250e-01   3.030e-02   1.445e+01   2.97
  3   6.250e-02   2.643e-03   4.569e+01   3.52
  4   3.125e-02   1.723e-04   1.462e+02   3.94
  5   1.563e-02   1.076e-05   1.818e+02   4.00
  6   7.813e-03   6.637e-07   1.949e+02   4.02
  7   3.906e-03   4.107e-08   1.911e+02   4.01
  8   1.953e-03   2.552e-09   1.850e+02   4.01
  9   9.766e-04   1.588e-10   1.825e+02   4.01
 10   4.883e-04   9.668e-12   2.265e+02   4.04


### Exponential multistep version 2; 3 steps; exact starting values; one correction step

In [18]:
set!(psi, soliton, t0)
method = ExponentialMultistep(3, iters=1, version=2, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   4.537e-01
  2   1.250e-01   1.058e-01   8.349e+00   2.10
  3   6.250e-02   1.086e-02   9.754e+01   3.28
  4   3.125e-02   2.048e-03   8.604e+00   2.41
  5   1.563e-02   2.102e-04   1.797e+02   3.28
  6   7.813e-03   1.475e-05   1.765e+03   3.83
  7   3.906e-03   9.275e-07   3.785e+03   3.99
  8   1.953e-03   5.729e-08   4.376e+03   4.02
  9   9.766e-04   3.546e-09   4.301e+03   4.01
 10   4.883e-04   2.202e-10   4.154e+03   4.01


### Exponential multistep version 2; 3 steps; starting values by Suzuki composition method; one correction step

In [19]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
#starting_method = SplittingMethod(a,b)
starting_method = CompositionMethod(g,3)
method = ExponentialMultistep(3, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   4.319e-01
  2   1.250e-01   1.053e-01   7.258e+00   2.04
  3   6.250e-02   1.086e-02   9.625e+01   3.28
  4   3.125e-02   2.048e-03   8.570e+00   2.41
  5   1.563e-02   2.102e-04   1.798e+02   3.28
  6   7.813e-03   1.475e-05   1.765e+03   3.83
  7   3.906e-03   9.275e-07   3.785e+03   3.99
  8   1.953e-03   5.729e-08   4.376e+03   4.02
  9   9.766e-04   3.546e-09   4.301e+03   4.01
 10   4.883e-04   2.202e-10   4.153e+03   4.01


### Exponential multistep version 2; 4 steps; starting values by Suzuki composition method; one correction step

In [20]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   4.768e-01
  2   1.250e-01   9.169e-02   1.289e+01   2.38
  3   6.250e-02   1.203e-02   4.056e+01   2.93
  4   3.125e-02   1.326e-03   8.155e+01   3.18
  5   1.563e-02   5.244e-05   1.372e+04   4.66
  6   7.813e-03   1.204e-06   3.576e+05   5.44
  7   3.906e-03   3.721e-08   4.477e+04   5.02
  8   1.953e-03   1.340e-09   1.315e+04   4.80
  9   9.766e-04   4.560e-11   2.188e+04   4.88
 10   4.883e-04   1.607e-12   1.542e+04   4.83


### Exponential multistep version 2; 5 steps; starting values by Suzuki composition method; one correction step

In [21]:
set!(psi, soliton, t0)
method = ExponentialMultistep(5, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p
-----------------------------------------------
  1   2.500e-01   4.153e-02
  2   1.250e-01   1.057e-01   6.411e-03  -1.35
  3   6.250e-02   1.422e-02   4.348e+01   2.89
  4   3.125e-02   9.562e-04   6.944e+02   3.89
  5   1.563e-02   2.730e-05   5.044e+04   5.13
  6   7.813e-03   7.407e-07   6.837e+04   5.20
  7   3.906e-03   1.416e-08   7.927e+05   5.71
  8   1.953e-03   2.227e-10   3.785e+06   5.99
  9   9.766e-04   3.491e-12   3.894e+06   6.00
 10   4.883e-04   3.393e-13   4.650e-02   3.36
